# Displaying beam based aligmennt measurement re

In [ ]:
import pathlib

from IPython.display import display, Markdown, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))
from pathlib import Path, PosixPath

In [ ]:
import xarray as xr
import numpy as np
import os.path
import matplotlib.pyplot as plt

from dataclasses import dataclass
# %matplotlib notebook
%matplotlib inline

In [ ]:
data_dir = Path(os.getcwd()) / ".." / "bact_analysis_bessyii"/ "scripts"
data_dir = Path(os.path.normpath(data_dir))
data_dir

In [ ]:
os.listdir(data_dir)

In [ ]:
# campain 1788
uids = [
    # variable names without dt ....
    # 'bceb0543-46f4-4bf8-9757-2340cacefb3b',
    # 'b61e48bb-6f7b-4559-9053-f6049fc51bac'
]
uids += ['7c58f6dd-92ce-407c-8b99-006fc532a8bb'
]
#campain 1945

In [ ]:
uid = uids[-1]
estimated_angles = xr.load_dataset(os.path.join(data_dir, f'estimated_angles_{uid}.nc'))
offsets = xr.load_dataarray(os.path.join(data_dir, f'offsets_{uid}.nc'))

In [ ]:
chck_path = '/home/waheed/workspace/bact/custom/bessyii/analysis/bact_analysis_bessyii/scripts'
os.listdir(chck_path)

In [ ]:
estimated_angles.orbit

In [ ]:

estimated_angles

In [ ]:
estimated_angles.orbit.attrs['theta']

In [ ]:
estimated_angles.fit_params.sel(name="q1m1d1r", plane="x", result="value", parameter="scaled_angle")

In [ ]:
pscale = 1000

In [ ]:
fig, ax = plt.subplots(1, 1)
ax.plot(
    estimated_angles.ds, estimated_angles.orbit.sel(name="q1m1d1r", plane="x") * pscale
)
ax.set_ylabel("dx [mm]")
ax.set_xlabel("ds [m]")

In [ ]:
fig, axes = plt.subplots(2, 1, figsize=[12, 9], sharex=True)
ax_x, ax_y = axes
a_scale = 1e6

for uid in uids:
    estimated_angles = xr.load_dataset(os.path.join(data_dir, f'estimated_angles_{uid}.nc'))
    offsets = xr.load_dataarray(os.path.join(data_dir, f'offsets_{uid}.nc'))
    
    ds = estimated_angles.ds_elems
    quad_names = estimated_angles.coords['name'].values
    
    ax_x.errorbar(
        ds,
        offsets.sel(name=quad_names, result="value", plane="x") * a_scale,
        yerr=offsets.sel(name=quad_names, result="error", plane="x") * a_scale,
        fmt=".-",
    )
    ax_y.errorbar(
        ds,
        offsets.sel(name=quad_names, result="value", plane="y") * a_scale,
        yerr=offsets.sel(name=quad_names, result="error", plane="y") * a_scale,
        fmt=".-",
    )
    ax_y.set_xlabel("s [m]")
    ax_x.set_xlabel("s [m]")
    ax_x.set_ylabel("x [$\mu$m]")
    ax_y.set_ylabel("y [$\mu$m]")

In [ ]:
@dataclass
class ReferenceOffsets:
    x: object
    xerr: object
    y: object
    yerr: object
    ds: object

In [ ]:
fig, axes = plt.subplots(2, 1, figsize=[12, 9], sharex=True)
ax_x, ax_y = axes
a_scale = 1e6

ref = None
for uid in uids:
    estimated_angles = xr.load_dataset(os.path.join(data_dir, f'estimated_angles_{uid}.nc'))
    offsets = xr.load_dataarray(os.path.join(data_dir, f'offsets_{uid}.nc'))
    
    
    ds = estimated_angles.ds_elems
    quad_names = estimated_angles.coords['name'].values
    x = offsets.sel(name=quad_names, result="value", plane="x") * a_scale
    y = offsets.sel(name=quad_names, result="value", plane="y") * a_scale
    xerr = offsets.sel(name=quad_names, result="error", plane="x") * a_scale
    yerr = offsets.sel(name=quad_names, result="error", plane="x") * a_scale
    if not ref:
        xerr= np.absolute(xerr)
        yerr= np.absolute(yerr)
        ref = ReferenceOffsets(x=x, y=y, xerr=xerr, yerr=yerr, ds=ds)
        print("Set refence plots start after next plot")
        continue 
    else:
        xerr = np.absolute(xerr) + ref.xerr
        yerr = np.absolute(xerr) + ref.yerr
        
    ax_x.errorbar(
        ds,
        x - ref.x,
        yerr=xerr,
        fmt=".-"
    )
    ax_y.errorbar(
        ds,
        y - ref.y,
        yerr=yerr,
        fmt=".-",
    )
    ax_y.set_xlabel("s [m]")
    ax_x.set_xlabel("s [m]")
    ax_x.set_ylabel("x [$\mu$m]")
    ax_y.set_ylabel("y [$\mu$m]")